In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,46507
2,Huelva,Confirmados PDIA 14 días,437
3,Huelva,Tasa PDIA 14 días,"85,15696552799267"
4,Huelva,Confirmados PDIA 7 días,217
5,Huelva,Tasa PDIA 7 dias,"42,286181966989496"
6,Huelva,Total Confirmados,46710
7,Huelva,Curados,45361
8,Huelva,Fallecidos,420


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  46507.0


/tmp/ipykernel_73404/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12794.0


/tmp/ipykernel_73404/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_73404/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_73404/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_73404/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1893 personas en los últimos 7 días 

Un positivo PCR cada 1042 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,46507.0,217.0,437.0,513170.0,42.286182,85.156966,49.0
Huelva-Costa,27504.0,123.0,253.0,289548.0,42.480003,87.377568,36.0
Huelva (capital),12794.0,76.0,138.0,143837.0,52.837587,95.941934,20.0
Condado-Campiña,14460.0,73.0,152.0,156231.0,46.725682,97.291831,11.0
Cartaya,2162.0,17.0,50.0,20083.0,84.648708,248.966788,6.0
Bollullos Par del Condado,1317.0,19.0,20.0,14387.0,132.063669,139.014388,5.0
Isla Cristina,3182.0,14.0,25.0,21393.0,65.441967,116.860655,5.0
Lepe,3056.0,8.0,12.0,27880.0,28.694405,43.041607,3.0
Moguer,2045.0,6.0,33.0,21867.0,27.438606,150.912334,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campofrío,58.0,5.0,8.0,713.0,701.262272,1122.019635,0.0
Almonaster la Real,92.0,6.0,7.0,1788.0,335.570470,391.498881,1.0
Palos de la Frontera,1097.0,20.0,32.0,11742.0,170.328734,272.525975,1.0
Villanueva de las Cruces,21.0,1.0,1.0,387.0,258.397933,258.397933,0.0
Cartaya,2162.0,17.0,50.0,20083.0,84.648708,248.966788,6.0
San Bartolomé de la Torre,389.0,1.0,8.0,3761.0,26.588673,212.709386,NaN
Minas de Riotinto,189.0,1.0,6.0,3812.0,26.232949,157.397692,0.0
Moguer,2045.0,6.0,33.0,21867.0,27.438606,150.912334,2.0
Bollullos Par del Condado,1317.0,19.0,20.0,14387.0,132.063669,139.014388,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,389.0,1.0,8.0,3761.0,26.588673,212.709386,0.0,0.125000
San Juan del Puerto,770.0,1.0,7.0,9411.0,10.625863,74.381043,0.0,0.142857
Ayamonte,2113.0,1.0,7.0,21104.0,4.738438,33.169067,0.0,0.142857
Trigueros,546.0,1.0,6.0,7862.0,12.719410,76.316459,0.0,0.166667
Minas de Riotinto,189.0,1.0,6.0,3812.0,26.232949,157.397692,0.0,0.166667
Moguer,2045.0,6.0,33.0,21867.0,27.438606,150.912334,2.0,0.181818
Aljaraque,1649.0,3.0,14.0,21474.0,13.970383,65.195120,0.0,0.214286
Cartaya,2162.0,17.0,50.0,20083.0,84.648708,248.966788,6.0,0.340000
Gibraleón,952.0,2.0,5.0,12737.0,15.702285,39.255712,0.0,0.400000
